In [ ]:
import numpy as np

In [ ]:
class FuzzyVariable:
    def __init__(self):
        self.labels = {}
        self.value = 0

    def toDiscrete(self):
        ret = {}
        graph = self.labels
        value = self.value
        for key in graph.keys():
            for i in range(len(graph[key]) - 1):
                if (graph[key][i][0] <= value and value <= graph[key][i + 1][0]):
                    if (graph[key][i][0] == -np.inf):
                        ret[key] = graph[key][i][1]
                        continue
                    if (graph[key][i + 1][0] == np.inf):
                        ret[key] = graph[key][i + 1][1]
                        continue
                    deltaY = graph[key][i + 1][1] - graph[key][i][1]
                    deltaX = graph[key][i + 1][0] - graph[key][i][0]
                    ret[key] = graph[key][i][1] + ((value - graph[key][i][0]) / deltaX) * deltaY
        return ret

In [ ]:
class Texture(FuzzyVariable):
    def __init__(self, value):
        self.value = value
        self.labels = {
            'very soft': [(-np.inf, 1.0), (0.2, 1.0), (0.4, 0), (np.inf, 0.0)],
            'soft': [(-np.inf, 0.0), (0.2, 0.0), (0.4, 1.0), (0.8, 0.0), (np.inf, 0.0)],
            'normal': [(-np.inf, 0.0), (0.3, 0.0), (0.7, 1.0), (0.9, 0.0), (np.inf, 0.0)],
            'resistant': [(-np.inf, 0.0), (0.7, 0.0), (0.9, 1.0), (np.inf, 1.0) ]
        }

In [ ]:
class Capacity(FuzzyVariable):
    def __init__(self, value):
        self.value = value
        self.labels = {
            'small': [(-np.inf, 1.0), (1.0, 1.0), (2.0, 0.0), (np.inf, 0.0)],
            'medium': [(-np.inf, 0.0), (2.5, 1.0), (4.0, 0.0), (np.inf, 0.0)],
            'high': [(-np.inf, 0.0), (3.0, 0.0), (4.0, 1.0), (np.inf, 1.0)]
        }

In [ ]:
class Ruler:
    def __init__(self):
        self.rules = {
           'very soft': {
                'small': 'delicate',
                'medium': 'easy',
                'high': 'normal'
            },
            'soft': {
                'small': 'easy',
                'medium': 'normal',
                'high': 'normal'
            },
            'normal': {
                'small': 'easy',
                'medium': 'normal',
                'high': 'intense'
            },
            'resistant': {
                'small': 'easy',
                'medium': 'small',
                'high': 'small'
            }
        }
        
    def evaluate(self, t, c):
        txtDiscrete = t.toDiscrete()
        capDiscrete = c.toDiscrete()
        resDisc = {}
        print(txtDiscrete)
        print(capDiscrete)
        for tKey, tValue in txtDiscrete.items():
            for cKey, cValue in capDiscrete.items():
                res = self.rules[tKey][cKey]
                val = min(tValue, cValue)
                if res in resDisc:
                    resDisc[res] = max(resDisc[res], val)
                else:
                    resDisc[res] = val
        return resDisc

In [ ]:
class Controller:
    def __init__(self, texture, capacity):
        self.rules = Ruler()
        self.txt = Texture(texture)
        self.cap = Capacity(capacity)

    def solve(self):
        agg = self.rules.evaluate(self.txt, self.cap)
        print(sorted(list(agg.items()), key = lambda x: x[1])[-1][0])

In [ ]:
c = Controller(0.2, 0.5)
c.solve()